In [1]:
from data_preprocessing import DataProcessor
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
import pandas as pd

In [10]:
def get_cv_results(clf,kf,inputDF,df):
    
    def print_pretty(title,scores):
        print title,':'
        print 'Average\tStandard Deviation'
        print np.mean(scores),'\t',np.std(np.mean(scores,axis=1))
        print 'CV mean scores:'
        print np.mean(scores,axis=1)
        print 'Mean per class:'
        print 'P1\tP2\tP3\tP4\tP5'
        for i in np.mean(scores,axis=0):
            print i,
        print
        #print 'CV:'
        #for f in scores:
        #    print f
        return
    
    cvscores = []
    p_scores = []
    r_scores = []
    for train,test in kf.split(inputDF,df['Priority']):
        train_input = inputDF[train]
        test_input = inputDF[test]
        train_output = df['Priority'][train]
        test_output = df['Priority'][test]
        clf.fit(train_input,train_output)
        prediction = clf.predict(test_input)
        scores = f1_score(test_output, prediction, average=None)
        p_scores.append(precision_score(test_output, prediction, average=None))
        r_scores.append(recall_score(test_output, prediction, average=None))
        cvscores.append(scores)
    print 'Cross-Validation Results'
    print_pretty('F1 Scores',cvscores)
    print ''
    print_pretty('Precision Scores',p_scores)
    print ''
    print_pretty('Recall Scores',r_scores)
    return cvscores, p_scores, r_scores

In [15]:
def get_test_result(clf, trainDF, df, testDF, df1):
    
    def print_pretty(title,score):
        print title,':'
        print 'Average:',np.mean(score)
        print 'P1\tP2\tP3\tP4\tP5'
        print score
        return

    clf.fit(trainDF, df['Priority'])
    prediction = clf.predict(testDF)
    f1 = f1_score(df1['Priority'], prediction, average=None)
    p = precision_score(df1['Priority'], prediction, average=None)
    r = recall_score(df1['Priority'], prediction, average=None)
    tempdf = pd.DataFrame()
    tempdf['F1'] = f1
    tempdf['Precision'] = p
    tempdf['Recall'] = r
    print tempdf
    print list(tempdf.mean(axis=0))
    #print 'TEST RESULTS:'
    #print_pretty('F1',f1)
    #print ''
    #print_pretty('Precision',p)
    #print ''
    #print_pretty('Recall',r)
    return f1,p,r

In [4]:
df_train = pd.read_csv('../data/processed/train_processed.csv')
df_test = pd.read_csv('../data/processed/test_processed.csv')

In [5]:
p = DataProcessor()
inputDF, df = p.fit_transform(df_train)
testDF, df1 = p.transform(df_test)

removed nan values
count vectorizer finished fitting
count vector finished transforming
dummy variables created
created a sparse matrix of all features
count vector finished transforming
dummy variables created
created a sparse matrix of all features


In [25]:
rf = RandomForestClassifier(n_estimators=19, min_samples_split=13, min_samples_leaf=1, class_weight="balanced", n_jobs=-1)
#nb = MultinomialNB()
#svc = SVC()
kf = KFold(n_splits=10)

## Cross-validation Results

In [26]:
%time f1, precision, recall = get_cv_results(rf,kf,inputDF,df)

Cross-Validation Results
F1 Scores :
Average	Standard Deviation
0.496118087028 	0.0125687086021
CV mean scores:
[ 0.49608219  0.50431894  0.52555944  0.50603347  0.4889616   0.48112999
  0.49097611  0.48307772  0.48756444  0.49747697]
Mean per class:
P1		P2	P3		P4		P5
0.397073609118 0.339465637432 0.91706674256 0.3589847544 0.467999691631

Precision Scores :
Average	Standard Deviation
0.498147697259 	0.0159003257469
CV mean scores:
[ 0.49424806  0.51984757  0.52397962  0.51996737  0.48576083  0.48119344
  0.49264439  0.48081955  0.4870852   0.49593094]
Mean per class:
P1		P2	P3		P4		P5
0.356397310414 0.356014362667 0.916147034047 0.386689727644 0.475490051523

Recall Scores :
Average	Standard Deviation
0.498034064625 	0.0125699883324
CV mean scores:
[ 0.50104629  0.49521308  0.53006434  0.49687998  0.49565959  0.48271338
  0.49139667  0.48857809  0.49050051  0.50828872]
Mean per class:
P1		P2	P3		P4		P5
0.449097788429 0.324725842723 0.918001744279 0.336088009122 0.462256938571
CPU time

## Results on Test Data

In [27]:
%time _,_,_ = get_test_result(clf=rf,trainDF=inputDF,df=df,testDF=testDF,df1=df1)

         F1  Precision    Recall
0  0.380686   0.347826  0.420402
1  0.336700   0.348432  0.325733
2  0.915330   0.914073  0.916592
3  0.337963   0.377261  0.306080
4  0.476923   0.472561  0.481366
[0.48952051296992288, 0.49203053572221672, 0.49003451079498417]
CPU times: user 24.6 s, sys: 40 ms, total: 24.6 s
Wall time: 4.9 s


(20761, 40)